# Feature Extraction for Basic BCI - Attempt One

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import butter, lfilter, filtfilt

%matplotlib inline

## Load Data

In [2]:
sampling_rate = 128
path = '../data/preprocessed/'
data = pd.read_csv(path+'CarloAll.csv', ',')
data.head()

,Time,AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,id,label
0,0.000000,3699.102539,3653.846191,3678.076904,3715.384522,3635.384522,3653.974365,3677.435791,3667.435791,3682.820557,3555.128174,3622.051270,3696.794922,3625.897461,3446.153809,0,0
1,0.007812,3782.307617,3741.281982,3754.615478,3808.461426,3674.230713,3729.358887,3750.512939,3738.846191,3755.000000,3639.230713,3708.974365,3781.410156,3693.718018,3530.897461,0,0
2,0.015625,3745.256348,3714.487061,3716.923096,3766.025635,3701.794922,3693.205078,3715.512939,3707.179443,3721.923096,3609.615478,3677.307617,3749.102539,3665.128174,3500.000000,0,0
3,0.023438,3728.205078,3696.794922,3701.794922,3759.743652,3692.820557,3674.871826,3697.435791,3689.871826,3705.384522,3591.410156,3661.794922,3731.153809,3643.974365,3483.974365,0,0
4,0.031250,3640.128174,3598.333252,3615.641113,3665.897461,3551.538574,3594.230713,3614.230713,3605.769287,3624.358887,3507.051270,3555.384522,3627.948730,3556.538574,3386.025635,0,0


In [3]:
data.tail()

,Time,AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,id,label
166075,5.960938,4197.436035,4188.461426,4192.436035,4202.307617,4186.025879,4190.512695,4192.051269,4193.205078,4183.717773,4186.538574,4192.179688,4173.205078,4192.563965,4201.922852,199,-1
166076,5.968750,4196.282227,4190.512695,4189.358887,4202.051269,4182.307617,4189.871582,4192.436035,4193.461426,4183.717773,4186.410156,4195.897461,4171.410156,4194.358887,4199.871582,199,-1
166077,5.976562,4192.307617,4190.256348,4187.051269,4200.256348,4181.666504,4188.461426,4191.025879,4195.769043,4185.641113,4186.922852,4198.974121,4167.051269,4195.128418,4197.563965,199,-1
166078,5.984375,4186.922852,4187.307617,4187.051269,4197.948731,4183.974121,4186.666504,4189.615234,4199.871582,4188.589844,4187.692383,4198.846191,4163.077148,4193.717773,4196.410156,199,-1
166079,5.992188,4183.589844,4172.179688,4179.743652,4191.410156,4191.538574,4190.641113,4187.820312,4199.230957,4189.102539,4187.563965,4186.025879,4160.256348,4193.333496,4193.846191,199,-1


## Crop Data to size of 0.5 to 3.5 sec -  New Dataset

In [4]:
data_3sec = pd.DataFrame()

for name, dd in data.groupby('id'):
    mask = (dd.Time >= 0.5) & (dd.Time <= 3.5)
    data_3sec = pd.concat([data_3sec, dd[mask]], axis=0, ignore_index=True)

data_3sec.describe()

,Time,AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,id,label
count,77000.000000,77000.000000,77000.000000,77000.000000,77000.000000,77000.000000,77000.000000,77000.000000,77000.000000,77000.000000,77000.000000,77000.000000,77000.000000,77000.000000,77000.000000,77000.00000,77000.000000
mean,2.000000,4166.171279,4167.277244,4167.816793,4169.317648,4166.234472,4168.877289,4169.332211,4169.027181,4170.314751,4169.855553,4167.959633,4159.509510,4167.576712,4164.980713,99.50000,0.000000
std,0.868283,345.858311,381.656043,370.199086,352.211151,377.874109,365.890504,360.993518,369.232866,362.265888,375.598835,378.553321,366.466275,372.575981,369.599916,57.73468,0.774602
min,0.500000,1372.051269,845.897461,931.794861,998.205139,858.076904,935.384644,966.923096,934.615356,956.666687,826.282043,864.102539,881.666687,903.076904,973.717957,0.00000,-1.000000
25%,1.250000,4149.615234,4156.666504,4160.769043,4155.256348,4150.641113,4162.051269,4164.102539,4160.384766,4160.769043,4160.897461,4159.743652,4143.461426,4156.282227,4152.692383,49.75000,-1.000000
50%,2.000000,4183.205078,4184.230957,4184.230957,4184.743652,4185.512695,4185.256348,4185.192383,4185.641113,4186.153809,4186.025879,4183.589844,4163.974121,4183.974121,4183.846191,99.50000,0.000000
75%,2.750000,4202.051269,4198.846191,4200.000000,4199.615234,4196.666504,4198.205078,4197.692383,4202.307617,4202.692383,4203.333496,4200.512695,4186.538574,4202.211670,4202.692383,149.25000,1.000000
max,3.500000,7728.846191,7728.077148,7700.769043,7697.179688,7711.282227,7673.205078,7682.820312,7693.589844,7686.922852,7726.538574,7714.358887,7695.000000,7710.256348,7677.692383,199.00000,1.000000


In [5]:
data_3sec.head(3)

,Time,AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,id,label
0,0.500000,3810.769287,3782.307617,3781.794922,3836.025635,3747.820557,3795.769287,3791.281982,3771.153809,3787.692383,3777.435791,3727.307617,3783.589844,3712.435791,3582.692383,0,0
1,0.507812,3625.000000,3590.897461,3614.487061,3656.666748,3550.641113,3618.974365,3611.666748,3604.358887,3623.846191,3600.512940,3545.897461,3619.358887,3517.692383,3402.179443,0,0
2,0.515625,3521.153809,3480.897461,3510.897461,3547.435791,3465.256348,3518.461426,3513.205078,3499.358887,3522.179443,3491.281982,3444.487061,3513.205078,3411.025635,3295.641113,0,0


## Bandpass Filter ButterworthBandpass 
http://scipy.github.io/old-wiki/pages/Cookbook/ButterworthBandpass

mu (8-13Hz) and beta (14-30Hz)

In [17]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a
   
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def butter_bandpass_filtfilt(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return filtfilt(b, a, data) 


In [13]:
def highpass(data, BUTTER_ORDER=3, sampling_rate=10000, cut_off=500.0):
    Wn = (float(cut_off) / (float(sampling_rate) / 2.0), 0.95)
    b, a = butter(BUTTER_ORDER, Wn, 'pass')
    return filtfilt(b, a, data) 

In [18]:
mask = (data_3sec.id == 0)
dd = data_3sec[mask].AF3
dd = dd.to_xarray()

dd_mu = highpass(dd)

In [23]:
# TODO
lowcut, highcut, fs = 8, 13, sampling_rate
b, a = butter_bandpass(lowcut, highcut, fs)
# w, h = freqz(b, a, worN=2000)

mask = (data_3sec.id == 0)
dd = data_3sec[mask].AF3
dd = dd.to_xarray()
dd_mu = butter_bandpass_filter(dd, lowcut, highcut, fs)
# dd_mu

In [49]:
data_mupass = pd.DataFrame()
electrodes = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8']
lowcut, highcut, fs = 8, 13, sampling_rate
dat_el = {}

for name, dat in data_3sec.groupby('id'):
    for el in electrodes:
        d = butter_bandpass_filter(dat[el], 8, 13, sampling_rate)
        print(str(el))
        dat_a[str(el)] = np.log(np.var(d))
#     dat[electrodes] = dat[electrodes].apply(butter_bandpass(8, 13, sampling_rate))
#     dat_a = butter_bandpass_filter(dat, 8, 13, sampling_rate)
#     data_mupass = pd.concat([data_mupass, dat], axis=0, ignore_index=True)
    
dat_mu
# print(dat_a.shape)
# data_mupass.describe()

AF3


TypeError: 'numpy.float64' object does not support item assignment